Importando todas as bibliotecas e bases de dados de abono e aposentadoria:

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()

In [2]:
from script_tratamento_abono import tratamento_abono

time_series_abono = []

for year in ['2017', '2018', '2019', '2020', '2021']:
    for month in ['01', '07']:
        filename = f"{month}_{year}"
        base = tratamento_abono(filename)
        time_series_abono.append(base)

In [3]:
from script_tratamento_aposentados import tratamento_aposentados

time_series_aposentadoria = []
filenames = ['01_2017', '07_2017', '01_2018', '07_2018','01_2019', '06_2019', '05_2020', '07_2021', '11_2021']

for file in filenames:
    df = tratamento_aposentados(file)
    time_series_aposentadoria.append(df)

## Cruzando as bases de dados

Cruzando a base de dados de abono com a de aposentadoria, obtendo uma base de pessoas que receberam abono e já se aposentaram

In [ ]:
df = time_series_abono[0]
aposentadoria_df_completo = pd.concat(time_series_aposentadoria)
full_matched_df = df.merge(aposentadoria_df_completo, on="CPF")

Depois, extraímos somente as colunas que nos interessam e exportamos o csv

In [35]:
feature_names = ['nome_x', 'escolaridade', 'UF_residencia', 'anos_servico', 'valor_abono', 'valor_aposentadoria']
full_matched_df = full_matched_df.filter(items=feature_names)
full_match_df = full_matched_df.rename(columns={'nome_x': 'nome'})

In [38]:
full_matched_df.to_csv("servidores.csv")